In [1]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow import keras
import os
import datetime
from gym import wrappers
from particle_envs.make_env import make_env

In [2]:
class MyModel(tf.keras.Model):
    def __init__(self, num_states, hidden_units, num_actions):
        super(MyModel, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(input_shape=(num_states,))
        self.hidden_layers = []
        for output in hidden_units:
            self.hidden_layers.append(Dense(output, activation='relu', kernel_initializer='RandomNormal'))
        self.output_layer = Dense(num_actions, activation='linear', kernel_initializer='RandomNormal')
        
    @tf.function
    def call(self, inputs):
        z = self.input_layer(inputs)
        for layer in self.hidden_layers:
            z = layer(z)
        output = self.output_layer(z)
        return output

In [3]:
class DQN:
    def __init__(self, num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr):
        self.num_actions = num_actions
        self.batch_size = batch_size
        self.optimizer = tf.optimizers.Adam(lr)
        self.gamma = gamma
        self.model = MyModel(num_states, hidden_units, num_actions)
        self.experience = {'s': [], 'a': [], 'r': [], 's2': [], 'done': []}  # done (bool): is current state terminal
        self.max_experiences = max_experiences
        self.min_experiences = min_experiences
         
    def predict(self, inputs):
        return self.model(np.atleast_2d(inputs.astype('float32')))
    
    def train(self, TargetNet):
        if len(self.experience['s']) < self.min_experiences:
            return 0, 0
        ids = np.random.randint(low=0, high=len(self.experience['s']), size=self.batch_size)
        states = np.asarray([self.experience['s'][i] for i in ids])
        actions = np.asarray([self.experience['a'][i] for i in ids])
        rewards = np.asarray([self.experience['r'][i] for i in ids])
        states_next = np.asarray([self.experience['s2'][i] for i in ids])
        dones = np.asarray([self.experience['done'][i] for i in ids])
        value_next = np.max(TargetNet.predict(states_next), axis=1)
        actual_values = np.where(dones, rewards, rewards + self.gamma*value_next)
        
        with tf.GradientTape() as tape:
            selected_action_values = tf.math.reduce_sum(
                self.predict(states) * actions)
            loss = tf.math.reduce_mean(tf.square(actual_values - selected_action_values))
    
        variables = self.model.trainable_variables
        gradients = tape.gradient(loss, variables)
        self.optimizer.apply_gradients(zip(gradients, variables))
        return loss, selected_action_values
        
    def get_action(self, states, epsilon):
        if np.random.random() < epsilon:
            return np.random.choice(self.num_actions)
        else:
            return np.argmax(self.predict(np.atleast_2d(states))[0])


    def add_experience(self, exp):
        if len(self.experience['s']) >= self.max_experiences:
            for key in self.experience.keys():
                self.experience[key].pop(0)
        for key, value in exp.items():
            self.experience[key].append(value)

    def copy_weights(self, TrainNet):
        variables1 = self.model.trainable_variables
        variables2 = TrainNet.model.trainable_variables
        for v1, v2 in zip(variables1, variables2):
            v1.assign(v2.numpy())

In [4]:
def play_game(env, TrainNet, TargetNet, epsilon, copy_step):
    rewards = 0
    iters = 0
    done = False
    observations = env.reset()[0]
    losses = list()
    episode_length = 200
    for i in range(episode_length):
        env.render()
        action = TrainNet.get_action(observations, epsilon)
        # Convert action to one-hot
        action = [np.identity(env.action_space[0].n, dtype=int)[action]]
        prev_observations = observations
        observations, reward, done, _ = env.step(action)
        observations, reward, done = observations[0], reward[0], done[0]
        rewards += reward
        if done:
            env.reset()

        exp = {'s': prev_observations, 'a': action, 'r': reward, 's2': observations, 'done': done}
        TrainNet.add_experience(exp)
        loss, selected_action_values = TrainNet.train(TargetNet)
        if isinstance(loss, int):
            losses.append(loss)
        else:
            losses.append(loss.numpy())
        iters += 1
        if iters % copy_step == 0:
            TargetNet.copy_weights(TrainNet)
    return rewards, np.mean(losses), np.mean(selected_action_values)

In [5]:
def make_video(env, TrainNet):
    env = wrappers.Monitor(env, os.path.join(os.getcwd(), "videos"), force=True)
    rewards = 0
    steps = 0
    done = False
    observation = env.reset()
    observation = observation[0]
    for i in range(100):
#         env.render()
        action = TrainNet.get_action(observation, 0)
        # Convert action to one-hot
        action = [np.identity(env.action_space[0].n, dtype=int)[action]]
        observation, reward, done, _ = env.step(action)
        observation, reward, done = observation[0], reward[0], done[0]
        steps += 1
        rewards += reward
    print("Testing steps: {} rewards {}: ".format(steps, rewards))

In [6]:
def main():
    env = make_env('simple')
    gamma = 0.95
    copy_step = 1
    num_states = len(env.observation_space[0].sample())
    num_actions = env.action_space[0].n
    hidden_units = [24, 24]
    max_experiences = 100000
    min_experiences = 100
    batch_size = 32
    lr = 1e-3
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    test_model_path = './dqn_model/model_{}'.format(current_time)
    log_dir = 'logs/dqn/' + current_time
    summary_writer = tf.summary.create_file_writer(log_dir)
    TrainNet = DQN(num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr)
    TargetNet = DQN(num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr)
    N = 10
    total_rewards = np.empty(N)
    epsilon = 0.99
    decay = 0.995
    min_epsilon = 0.1
    for n in range(N):
        epsilon = max(min_epsilon, epsilon * decay)
        total_reward, losses, action_values_avg = play_game(env, TrainNet, TargetNet, epsilon, copy_step)
        total_rewards[n] = total_reward
        avg_rewards = total_rewards[max(0, n - 100):(n + 1)].mean()
        with summary_writer.as_default():
            tf.summary.scalar('episode reward', total_reward, step=n)
            tf.summary.scalar('running avg reward(100)', avg_rewards, step=n)
            tf.summary.scalar('average loss)', losses, step=n)
            tf.summary.scalar('average Q value', action_values_avg, step=n)
        if n % 9 == 0:
            print("episode:", n, "episode reward:", total_reward, "epsilon:", epsilon, "avg reward (last 100):", avg_rewards,
                  "episode loss: ", losses)
            TrainNet.model.save(test_model_path)
    print("avg reward for last 100 episodes:", avg_rewards)
#     make_video(env, TrainNet)

    TrainNet.model.save(test_model_path)
    env.close()

In [7]:
if __name__ == '__main__':
    main()

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


episode: 0 episode reward: 289.3912869621781 epsilon: 0.98505 avg reward (last 100): 289.3912869621781 episode loss:  11.520329580307006
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: .\dqn_model\model_20200719-181628\assets
episode: 9 episode reward: 229.62983814065802 epsilon: 0.9415990291611142 avg reward (last 100): 216.77800407716464 episode loss:  24.004927
INFO:tensorflow:Assets written to: .\dqn_model\model_20200719-181628\assets
avg reward for last 100 episodes: 216.77800407716464
INFO:tensorflow:Assets written to: .\dqn_model\model_20200719-181628\assets


In [1]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow import keras
import os
import datetime
from gym import wrappers
from particle_envs.make_env import make_env
import time

env = make_env('simple')
TestModel = keras.models.load_model('C:\\Parth\\CS Projects\\multiagent-rl\\dqn_model\\model_20200718-162721', compile=False)
for i in range(10):
    obs = env.reset()
    obs = obs[0]
    print('Episode {}:'.format(i+1))
    for j in range(200):
        env.render()
        action = np.argmax(TestModel(np.atleast_2d(obs.astype('float32')))[0])
        action = [np.identity(env.action_space[0].n, dtype=int)[action]]
        obs, reward, done, info = env.step(action)
        print(f'Step {j}:',action, obs, reward, done, info)
        obs, done = obs[0], done[0]
        time.sleep(0.5)
        if done:
            print("Out of boundary")
            break
env.close()

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Episode 1:
Step 0: [array([0, 0, 1, 0, 0])] [array([-0.5       ,  0.        , -0.03335469,  0.83513758])] [-0.6985673129855177] [False] {'n': [{}]}
Step 1: [array([0, 0, 1, 0, 0])] [array([-0.875     ,  0.        ,  0.05414531,  0.83513758])] [-0.7003864928309004] [False] {'n': [{}]}
Step 2: [array([0, 0, 1, 0, 0])] [array([-1.15625   ,  0.        ,  0.16977031,  0.83513758])] [-0.7262767371801561] [False] {'n': [{}]}
Step 3: [array([0, 0, 1, 0, 0])] [array([-1.3671875 ,  0.        ,  0.30648906,  0.83513758])] [-0.7913903238526291] [False] {'n': [{}]}
Step 4: [array([0, 0, 1, 0, 0])] [array([-1.52539062,  0.        ,  0.45902813,  0.83513758])] [-0.9081615982367532] [False] {'n': [{}]}
Step 5: [array([0, 0, 1, 0, 0])] [array([-1.64404297,  0.        ,  0.62343242,  0.83513758])] [-1.0861227636373125] [False] {'n': [{}]}
Step 6: [array([0, 0, 1, 0, 0])] [array([-1.73303223,  0.        ,  0.79673565,  0.83513758])] [-1.3322424665717145] [True] {'n': [{}]}
Out of boundary
Episode 2:
Step

KeyboardInterrupt: 